<a href="https://colab.research.google.com/github/mparker2103/PatternFlow/blob/topic-recognition/recognition/s4436238_UNet/Improved_UNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LeakyReLU, BatchNormalization, Dense, Dropout, UpSampling2D, Reshape, concatenate, add
from tensorflow.keras.models import Model
from keras import backend as k

In [6]:
depth = 16
n = 256
batch_size = 20
epochs = 20

In [8]:
# create the network
def UNet():
    Inputs = Input(shape=(n,n,3), name='Inputs')

    # Downsample 1
    DownConv1_0 = Conv2D(depth, (3,3), padding='same', activation='relu', name='DownConv1_0')(Inputs)
    Batch1_0 = BatchNormalization(name='Batch1_0')(DownConv1_0)
    Relu1_0 = LeakyReLU(alpha=0.01, name='Relu1_0')(Batch1_0)
    Dropout1 = Dropout(0.3, name='Dropout1')(Relu1_0)
    Batch1_1 = BatchNormalization(name='Batch1_1')(Dropout1)
    Relu1_1 = LeakyReLU(alpha=0.01, name='Relu1_1')(Batch1_1)
    DownConv1_1 = Conv2D(depth, (3,3), padding='same', activation='relu', name='DownConv1_1')(Relu1_1)
    Pool1 = MaxPooling2D(pool_size=(2,2), name='Pool1')(DownConv1_1)
    Add1 = add(name='Add1')([DownConv1_0, Pool1])

    # Downsample 2
    DownConv2_0 = Conv2D(depth*2, (3,3), padding='same', activation='relu', name='DownConv2_0')(Add1)
    Batch2_0 = BatchNormalization(name='Batch2_0')(DownConv2_0)
    Relu2_0 = LeakyReLU(alpha=0.01, name='Relu2_0')(Batch2_0)
    Dropout2 = Dropout(0.3, name='Dropout2')(Relu2_0)
    Batch2_1 = BatchNormalization(name='Batch2_1')(Dropout2)
    Relu2_1 = LeakyReLU(alpha=0.01, name='Relu2_1')(Batch2_1)
    DownConv2_1 = Conv2D(depth, (3,3), padding='same', activation='relu', name='DownConv2_1')(Relu2_1)
    Pool2 = MaxPooling2D(pool_size=(2,2), name='Pool2')(DownConv2_1)
    Add2 = add(name='Add2')([DownConv2_0, Pool2])

    # Downsample 3
    DownConv3_0 = Conv2D(depth*4, (3,3), padding='same', activation='relu', name='DownConv3_0')(Add2)
    Batch3_0 = BatchNormalization(name='Batch3_0')(DownConv3_0)
    Relu3_0 = LeakyReLU(alpha=0.01, name='Relu3_0')(Batch3_0)
    Dropout3 = Dropout(0.3, name='Dropout3')(Relu3_0)
    Batch3_1 = BatchNormalization(name='Batch3_1')(Dropout3)
    Relu3_1 = LeakyReLU(alpha=0.01, name='Relu3_1')(Batch3_1)
    DownConv3_1 = Conv2D(depth, (3,3), padding='same', activation='relu', name='DownConv3_1')(Relu3_1)
    Pool3 = MaxPooling2D(pool_size=(2,2), name='Pool3')(DownConv3_1)
    Add3 = add(name='Add3')([DownConv3_0, Pool3])

    # Downsample 4
    DownConv4_0 = Conv2D(depth*8, (3,3), padding='same', activation='relu', name='DownConv4_0')(Add3)
    Batch4_0 = BatchNormalization(name='Batch4_0')(DownConv4_0)
    Relu4_0 = LeakyReLU(alpha=0.01, name='Relu4_0')(Batch4_0)
    Dropout4 = Dropout(0.3, name='Dropout4')(Relu4_0)
    Batch4_1 = BatchNormalization(name='Batch4_1')(Dropout4)
    Relu4_1 = LeakyReLU(alpha=0.01, name='Relu4_1')(Batch4_1)
    DownConv4_1 = Conv2D(depth, (3,3), padding='same', activation='relu', name='DownConv4_1')(Relu4_1)
    Pool4 = MaxPooling2D(pool_size=(2,2), name='Pool4')(DownConv4_1)
    Add4 = add(name='Add4')([DownConv4_0, Pool4])

    # Middle Layer
    MidConv1 = Conv2D(depth*16, (3,3), padding='same', activation='relu', name='MidConv1')(Add4)
    MidConv2 = Conv2D(depth*16, (3,3), padding='same', activation='relu', name='MidConv2')(MidConv1)

    # Upsample 4
    UpNet4 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet4')(MidConv2)
    UpConv4 = Conv2D(depth*8, (3,3), padding='same', activation='relu', name='UpConv4')(UpNet4)
    Batch4 = BatchNormalization(name='Batch4')(UpConv4)
    Relu4 = LeakyReLU(alpha=0.01, name='Relu4')(Batch4)
    Concat4 = concatenate([Relu4, DownConv4_1], axis=3, name='Concat4')

    # Upsample 3
    UpNet3 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet3')(Concat4)
    UpConv3 = Conv2D(depth*4, (3,3), padding='same', activation='relu', name='UpConv3')(UpNet3)
    Batch3 = BatchNormalization(name='Batch3')(UpConv3)
    Relu3 = LeakyReLU(alpha=0.01, name='Relu3')(Batch3)
    Concat3 = concatenate([Relu3, DownConv3_1], axis=3, name='Concat3')

    # Upsample 2
    UpNet2 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet2')(Concat3)
    UpConv2 = Conv2D(depth*2, (3,3), padding='same', activation='relu', name='UpConv2')(UpNet2)
    Batch2 = BatchNormalization(name='Batch2')(UpConv2)
    Relu2 = LeakyReLU(alpha=0.01, name='Relu2')(Batch2)
    Concat2 = concatenate([Relu2, DownConv2_1], axis=3, name='Concat2')

    # Upsample 1
    UpNet1 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet1')(Concat2)
    UpConv1 = Conv2D(depth, (3,3), padding='same', activation='relu', name='UpConv1')(UpNet1)
    Batch1 = BatchNormalization(name='Batch1')(UpConv1)
    Relu1 = LeakyReLU(alpha=0.01, name='Relu1')(Batch1)
    Concat1 = concatenate([Relu1, DownConv1_1], axis=3, name='Concat1')

    # Reshape into two dimensions
    dense = Dense(1, activation='softmax', name='Dense')(Concat1)
    Outputs = Reshape(test.shape[1:], name='Outputs')(dense)

    #Model
    model = Model(Inputs, Outputs, name='UNet')
    return model

In [9]:
# Define dice coefficient metric
def dice(y, y_pred):
    y_flat = k.flatten(y)
    y_pred_flat = k.flatten(y_pred) 
    
    intersect = k.sum(y_flat * y_pred_flat)
    sum = k.sum(k.square(y_flat)) + k.sum(k.square(y_pred_flat))
    return (2 * intersect) / sum